# Single Channel Artifact Removal

## Summary 
This notebook implements the algorithm proposed by [Maddirala and Veluvolu](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8155082/).
The algorithm works in the folowing steps:
1. Transform the single channel input signal (`x`) into a multivariate matrix (`X`) of `K` columns of `W` window length.
2. Compute the following features in Table 1 for each column (`k`):
3. Compute a k-means clustering algorithm with `n_clusters`.
4. Construct `n_clusters` component signals using the k-means clustering information and diagonal averaging the appropriate features.
5. Calculate the `fd` fractal dimension of the decomposed components. Label the components as artifacs (i.e., boolean vector = 1) if they are below `fd_threshold`.
    - Obtain a single artifact signal by multiplying the original input signal `x` by the artifact boolean.  
6. Apply singular spectrum analysis (SSA) to the artifact signal.
    - Subtract from the original signal (`x`) the estimated atifact signal obtained with SSA.

Table 1. Features calculated for matrix `X`.
Feature         | Equation
:---------------|:------------------------
Energy          | $$\text{Energy} = x^2$$
Hjorth mobility | $$\text{Hjorth mobility} = \sqrt{\frac{var(\frac{dy(t)}{dt})}{var(y(t))}}$$
Kurtosis        | $$\text{Kurtosis} = n * \frac{\sum_i^n (Y_i-\bar{Y}^4)}{\sum_i^n (Y_i-\bar{Y}^2)^2}$$
Amplitude range | $$\text{Range} = \text{max}(x) - \text{min}(x)$$

**Notes:**
- Main parameters are included in the first code segment
- Plotting booleans are included in each section

In [1]:
# Enable this for interactive plots
%matplotlib widget  

## Import libraries
import os
import time
import pyts as ts
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import scipy.linalg as linalg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy import matlib as matlib
from pyts.decomposition import SingularSpectrumAnalysis

complete_time = time.time()

## Parameters
window_length = 125     # Window length [n_samples]
n_clusters = 4          # Number of clusters for the k-means [n]
fd_threshold = 1.4      # Threshold value for the fractal dimension (FD) [A.U.]
ssa_threshold = 0.03    # Theshold value for SSA [%]
                        # eigenvalues > ssa_threshold are included in SSA
eeg_chan = 0            # EEG channel index or string name to use

# Path of data of interest
abs_path = os.getcwd()
data_path = '\\Data\\Convolved\\'
eeg_file = 'conv_mus_data.npz'  # String name of the file to load

## Import data
Import the data to work with. This notebook is designed to import the data generated by [03_Convolve_EEG](/03_Convolve_EEG.ipynb). We are particularly interested in the eye artifact data. This is similar to the "ground truth" data presented in the paper.

In [2]:
# Settings
plot_raw = False    # Boolean to plot raw data

# Load and separate data
data = np.load(abs_path+data_path+eeg_file, allow_pickle=True)
eeg = data['conv'][0]   # EEG data [V]  
chans = data['chans']   # Channels [str]
srate = data['srate']   # Sampling rate [Hz]
time_vect = np.linspace(0, np.size(eeg,0)/srate, np.size(eeg,0)) # Generate time vector based on srate [sec]

# Select single EEG channel
if type(eeg_chan) == int:
    eeg_single = eeg[:,eeg_chan]
elif type(eeg_chan) == str:
    eeg_single = eeg[:,np.where(eeg_chan==chans)].squeeze()

if plot_raw:
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_title('RAW data')
    ax.set_xlabel('Time [sec]')
    ax.set_ylabel('Amplitude [$\mu$V]')
    ax.plot(time_vect, eeg*1e6)
    ax.grid()
    plt.show()

## Embedding and feature extraction
After importing the data and selecting a `single_eeg` channel. Convert the `single_eeg` vector into a 2D matrix (embedding) and extract the corresponding features (i.e., energy, Hjorth mobility, Kurtosis, and range). 

In [3]:
single_art_time = time.time()  # Time to compute single channel artifact removal

# Embedding matrix
# - Determine embedding matrix sizes
N = len(eeg_single)         # Total number of points
M = window_length           # Number of rows
K = N - window_length + 1   # Number of columns

# - Create embedding matrix with the correspongding indices of the vector data
idx_col = np.arange(K).reshape(1,-1).repeat(M, axis=0)
idx_row = np.arange(M).reshape(-1,1).repeat(K, axis=1)
idx_mat = idx_col + idx_row

# - Create embedding matrix for eeg
eeg_embedded = eeg_single[idx_mat]

# Features matrix
# - Create features
f1 = np.sum(eeg_embedded**2, axis=0)            # Energy [V^2]
f2 = np.sqrt(np.var(np.diff(eeg_embedded,axis=0),axis=0) / np.var(eeg_embedded,axis=0)) # H_mobility
f3 = stats.kurtosis(eeg_embedded, axis=0)       # Kurtosis
f4 = eeg_embedded.max(0) - eeg_embedded.min(0)  # Range

# - Concatenate features
eeg_features = np.array((f1, f2, f3, f4))

## K-means classification
Apply a k-means cluster classifier of `n_clases` to the `eeg_features` matrix. Get labels for each column of the `eeg_features` matrix.

In [4]:
plot_features = False    # Boolean to plot the features selected

kmeans = KMeans(n_clusters=n_clusters).fit(eeg_features.T)

if plot_features:
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_title('EEG features')
    ax.set_xlabel('Energy')
    ax.set_ylabel('H$_{mobility}$')
    ax.set_zlabel('Kurtosis')
    ax.scatter(f1, f2, f3, c=kmeans.labels_)
    ax.grid()
    plt.show()

## Reconstruct components
Using the labels obtained with the k-means classifier, reconstruct an `n_clusters` number of `eeg_component` signals. 

> The $j^{th}$ column vector $x^j$ of $X$ is placed (copied) in the $j^{th}$ column of matrix $\bar{X_i}$ when the $j^{th} feature vector $f^j$ of matrix $X$ belongs to the $i^{th}$ cluster $C_i$; otherwise a vector with zeros is placed (copied) in that column of matrix $\bar{X_i}$.

\begin{equation}
\bar{x}_i^j = \left\{ \begin{matrix*}[l] x^i    & \text{if } f^j \in C_i, \{i=1,2,...,L\} \\ 
                                         0      & \text{if } f^j \notin C_i \end{matrix*} \right.
\end{equation}

Where: 
- $X$ = `eeg_embedded`
- $\bar{X_i}$ = `eeg_decomposed`

Lastly, reconstruct each `eeg_component` by taking the anti-diagonal average of each `eeg_decomposed` matrix. Note that there will be `n_cluster` eeg components.


In [5]:
plot_components = False  # Boolean to plot calculated EEG components

shape = np.shape(eeg_embedded)
eeg_decomposed = np.zeros((shape[0], shape[1], n_clusters)) # 3D matrix containing the decomposed EEGs [M,K,n_clusters]
eeg_component = np.zeros([n_clusters, len(eeg_single)])         # Matrix of n_clusters EEG components reconstructed from eeg_decomposed

# Create n_cluster matrices, with columns corresponding to their kmeans cluster
for i in range(n_clusters):
    copy_index = (kmeans.labels_==i)    # Determine columns to copy based on kmeans
    eeg_decomposed[:,copy_index,i] = eeg_embedded[:,copy_index]
    
    # Reconstruct signal from antidiagonal averages
    eeg_shape = np.shape(eeg_decomposed[:,:,i])     # Number of rows and columns
    eeg_flip = np.flip(eeg_decomposed[:,:,i], 1)    # Flip EEG matrix from left to right

    for j, k in enumerate(range(-eeg_shape[0]+1, eeg_shape[1])):
        eeg_component[i,-j] = np.mean(np.diag(eeg_flip, k=k))    # Calculate antidiagonal averages

if plot_components:
    fig = plt.figure()
    
    for i in range(n_clusters):
        ax = fig.add_subplot(n_clusters,1,i+1)
        if i == 0: ax.set_title('EEG components')
        if i == n_clusters-1: ax.set_xlabel('Time [sec]')
        ax.set_ylabel('Amplitude [$\mu$V]')
        ax.plot(time_vect, eeg_component[i,:])
        ax.grid()
    plt.show()

## Fractal dimension

The fractal dimension (FD) is calculated as detailed in: [A Procedure to Estimate the Fractal Dimension of Waveforms](https://arxiv.org/pdf/1003.5266v1.pdf). 

\begin{align}
    x^*_i &= \frac{x_i}{x_{max}} \\
    y^*_i &= \frac{y_i-y_{min}}{y_{max}-y_{min}} \\
    \phi \approx D &= 1 + \frac{ln(L)}{ln(2N')}
\end{align}

> Where $x_{max}$ is the maximum $x_i$ and $y_{min}$ and $y_{max}$ are the minimum and maximum $y_i$. The fractal dimension of the waveform ($\phi$) is then approximated by $D$; where $L$ is the length of the curve in the unit square and $N' = N-1$.


Then, a binary artifact template is created by creating a mask of the components whose `fd < fd_threshold`. The sample values associated with the artifact are ones, the sample values associated with the non-artifact regions are zeroes. The binary artifact template is multiplied with the input signal; This isolates the artifact.



In [6]:
# Preallocate and initialize
fd = np.zeros(n_clusters)       # Fractal dimensions, one per n_cluster
n = np.size(eeg_component,-1)   # Number of data samples

# Normalize EEG
x_norm = np.repeat(np.reshape(np.linspace(0, 1, n),[-1,1]),n_clusters,axis=1)
y_num = eeg_component - matlib.repmat(np.amin(eeg_component, axis=1, keepdims=True), 1, n)
y_den = matlib.repmat(np.amax(eeg_component, axis=1, keepdims=True) - np.amin(eeg_component, axis=1, keepdims=True), 1, n)
y_norm = np.divide(y_num, y_den).T
z = np.array([x_norm, y_norm]) # 3D Matrix to store x_norm and y_norm [sample x [x,y] x n_cluster]

# Calculate fractal dimension
l = np.sum(np.sum(np.abs(np.diff(z, axis=1)), axis=0), axis=0)  # Calculate length of signal (l1-norm for each n_cluster) [A.U.]
fd = 1 + np.log(l) / np.log(2*(n-1))

# Binary artifact creation
fd_mask = fd < fd_threshold # Apply threshold to FD to determine artifact components
eeg_mask = np.sum(eeg_component[fd_mask,:],0)    # Vector with artifact points != 0
eeg_artifact = eeg_single * (eeg_mask != 0).astype(int) # Multiply mask with original to get eeg_artifact with correct values [V]

## Singular Spectrum Analysis
Singular Spectrum Analysis (SSA) is applied to the `eeg_artifact` signal 

In [7]:
plot_final = False  # Boolean to plot final results

# Embedding
artifact_embedded = eeg_artifact[idx_mat]

# Singular Value Decomposition
[u, s, vh] = np.linalg.svd(artifact_embedded)

# Determine number of groups
eigen_ratio = (s / np.sum(s)) > ssa_threshold   # Keep only eigenvectors > ssa_threshold
vh_sub = vh[0:np.size(s)]
b = u[:,eigen_ratio] @ np.diag(s[eigen_ratio]) @ vh_sub[eigen_ratio,:]  # Reconstruct signal

# Reconstruct signal from antidiagonal averages
artifact_shape = np.shape(b)
artifact_flip = np.flip(b,1)

artifact_reconstruct = np.zeros(artifact_shape[0]+artifact_shape[1]-1)
for j, k in enumerate(range(-artifact_shape[0]+1, artifact_shape[1])):
    # Organize values from end to start to get right order
    artifact_reconstruct[-j] = np.mean(np.diag(artifact_flip, k=k))

# Subtract artifact signal from original to get clean_eeg
clean_eeg = eeg_single - artifact_reconstruct

# Plot original and cleaned EEG
if plot_final:
    fig, ax = plt.subplots()
    ax.plot(eeg_single)
    ax.plot(clean_eeg)
    ax.legend(['Contaminated', 'Clean'])
    ax.grid()

In [8]:
print(f"Complete time was {time.time() - complete_time:.{2}} sec")
print(f"Artifact removal time was {time.time() - single_art_time:.{2}} sec")

Complete time was 0.8 sec
Artifact removal time was 0.68 sec
